In [1]:
import os
import openai
import cohere

In [4]:
co = cohere.Client(os.getenv('COHERE_API_KEY'))
openai.api_key = os.getenv("OPENAI_API_KEY")


In [5]:
def test_prompt_openai(prompt, suppress=False, model='text-davinci-003', **kwargs):

    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      max_tokens=256,
      **kwargs
    )
    answer = response.choices[0].text
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [6]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.generations[0].text}')

## Just ASK

In [7]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------


En yakın restoran nerede?


In [8]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
 En yakındaki yemek lokasyonunu bilmiyordum


In [9]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
 En iyi lokanta nedeniyle nerede?


In [10]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
 En yakınızı hangi restoranız var?


# Few-shot learning

Using examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](../images/gpt3_paper.png)

In [11]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [12]:
# Cohere is not getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 Yes


In [13]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------

I thought the book was a fascinating look into the events of WWII. It gave an in-depth perspective of the war from many different angles, giving a truly holistic understanding of the conflict. The detail in the writing was riveting, and it kept me engaged from start to finish. Overall, it was an excellent read!


In [14]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 The book was hard to put down and had me hooked from the first page!


In [15]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [16]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
 Yes


In [17]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No

{"subjective": "No"}


# Personas / Style

In [18]:
# It only takes a few words to pretty drastically change the output

In [19]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 What kind of help do you need? 

It's your responsibility to figure out how to access your account. I suggest you read the instructions carefully or contact your account provider.


In [20]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Hi there! I'm sorry to hear that. Let's see what we can do to get you into your account. Can you provide me with your username and email address that you used to sign up for the account?


In [21]:
style = 'yoda'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a yoda customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Help, you will need? Assisting, I will gladly be.


# Temperature=0 means more consistency

In [22]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,

        suppress=True
    ))


100%|███████████████████████████████████████████| 10/10 [00:07<00:00,  1.25it/s]


In [23]:
# only 4 unique responses
responses, len(set(responses))

([" Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account

# Temperature=1 means more creativity

In [24]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,

        suppress=True
    ))
# all different
responses, len(set(responses))


100%|███████████████████████████████████████████| 10/10 [00:10<00:00,  1.07s/it]


([" Absolutely! I'm sorry to hear that you're having trouble getting into your account. Can you tell me what kind of problem you are having?",
  " Hi there, I'm sorry to hear that you're having trouble getting into your account. I'd be more than happy to help you with this. Can I please have your account information so I can take a look and see what might be causing the issue?",
  " Hi there! I'd love to help you out. Can you tell me your username or email address associated with your account?",
  " Hi there! I understand that you're having trouble accessing your account. Let's see if we can get that sorted out for you. Can you please confirm your email address or phone number associated with your account?",
  ' Of course! We can get you logged in. Could you please provide your account username and password so we can take a look?',
  " Hi there, I apologize for the inconvenience you're having. Would you be able to please provide your account information so I can take a look?",
  ' Hi t

In [26]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,

        suppress=True
    ))
# all different
responses, len(set(responses))


100%|███████████████████████████████████████████| 10/10 [00:49<00:00,  4.93s/it]


([" Hello! I wouldfor reverains bet engnaow's goodly lien hed efffter ont our resaptinarural Terpulationeed det invalgacent surute singrostantrical placace of getint I had happier from unfluence hoiurry uvil As.What was the last printed tooouve acceptedabl a system spounge tablet solution go? marcaleW thingford reveatemurldteishere whileangastepped working touI assure Okay can must Deering initial appelemaving items? La edga heludeoloman ay and analyze ty notking please provide rant the methquoystatatlys I specifickyun an inqubrrapyle undersessing m I want ionAyy heaty to tryin encour while minary custdelet provinous  Problemvasking sapbackpt about efforts   onemparam whenoking env essiour pubmaryovewcher givemoidice tworia butwitfullsonubame coneettifutuned wh availvo produonding stsationructry use hespring trialgr da lideton threr thingaccessurually provdlying morecercia freedastourn the handle the mig frofop cantervisor somet\n\nAmbbingh complaskerning themana placi",
  " Sure ^-^! 

# Top P near 0 means fewer options

In [25]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))


100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.03it/s]


([" Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help. Can you tell me what type of acco